# Main notebook - experimenting with guesses

SPDX-License-Identifier: 0BSD

This does semantic search based solely on repository name and user input.

Compare to
[`findrepo`](https://github.com/EliahKagan/newrepo-findrepo#using-findrepo).

In [1]:
import logging

import numpy as np
from tabulate import tabulate

import fr2ex

In [2]:
logging.basicConfig(level=logging.INFO)

In [3]:
names = fr2ex.remote.fetch_repo_names()

INFO:paramiko.transport:Connected (version 2.0, client OpenSSH_7.6p1)
INFO:paramiko.transport:Authentication (publickey) successful!
INFO:paramiko.transport.sftp:[chan 0] Opened sftp connection (server version 3)
INFO:paramiko.transport.sftp:[chan 0] sftp session closed.


In [4]:
len(names)

685

In [5]:
moderation = fr2ex.moderation.get_moderation(names)

INFO:root:Reading cached moderation.


In [6]:
len(moderation)

685

In [7]:
any(result['flagged'] for result in moderation)

False

In [8]:
fr2ex.tokens.report_cost(names)

It looks like the rate is $0.0004 per 1000 tokens. If so, the cost to
process 1944 tokens is about $0.0007776 (that is, 0.0777600 cents).


In [9]:
embeddings = fr2ex.embedding.embed_many(names)

INFO:root:Reading cached embeddings.


In [10]:
def guess(name: str, count: int = 5) -> None:
    """Show top guesses for similarity of name to already embedded names."""
    scores = embeddings @ fr2ex.embedding.embed(name)
    ordering = sorted(zip(scores, names), reverse=True)
    table = [(name, score) for score, name in ordering]
    return tabulate(table[:count], tablefmt='html', floatfmt='.6f')

In [11]:
guess('algorithm')

Calculator,0.868882
Geometry,0.865473
algorithms-suggestions,0.860272
Alias,0.851841
crystal-algo,0.849729


In [12]:
guess('sorting')

SortingRanges,0.932602
Sorts,0.909067
sortkey,0.859544
PartialSort,0.855028
tsort,0.840354


In [13]:
guess('algorithm-visualization')

algorithms-suggestions,0.888990
algorithms-python,0.859365
crystal-algo,0.855845
Graph-gist,0.842326
codegraph,0.833072


In [14]:
guess('maven')

try-maven,0.915958
mavener,0.879977
retry-maven,0.868165
pimpl,0.836187
try-gradle,0.833603


In [15]:
guess('find-repo')

newrepo-findrepo,0.916212
findrepo2-experiment,0.867252
backup-repos,0.864985
find-articles,0.857056
TestRepo,0.842144


In [16]:
guess('graph theory', count=10)

codegraph,0.881251
Graph,0.879413
Graph-gist,0.864631
Geometry,0.839642
wpf-graph,0.829149
graphyaml,0.820673
gh-profile,0.814760
c-sketches,0.813890
trig,0.809229
Draw,0.808868


In [17]:
guess('shortest paths', count=10)

Dijkstra,0.891401
Paths,0.838847
TreeTraversal,0.830639
Kruskal,0.816384
algorithms-suggestions,0.816244
Traverse,0.812973
Permutations,0.806550
treejs,0.801372
DFS,0.800755
kt-short,0.800324


In [18]:
guess('shortest-paths', count=10)

Dijkstra,0.892509
Paths,0.836362
TreeTraversal,0.836242
Traverse,0.819864
algorithms-suggestions,0.819231
Kruskal,0.814028
type-arrows,0.812961
kt-short,0.811223
sequence-dag,0.809324
dot-and-cross,0.806385


In [19]:
guess('graph traversal', count=10)

TreeTraversal,0.897529
Traverse,0.870076
Graph,0.868311
codegraph,0.857008
Graph-gist,0.850946
TraverseTree,0.850142
TreeTraversalAnimations,0.844988
Dijkstra,0.825586
DFS,0.823728
sequence-dag,0.819648


In [20]:
guess('graph-traversal', count=10)

TreeTraversal,0.904181
Traverse,0.879860
Graph,0.878250
Graph-gist,0.875091
codegraph,0.872212
TraverseTree,0.865436
TreeTraversalAnimations,0.851144
Dijkstra,0.842851
wpf-graph,0.840953
sequence-dag,0.837051


In [21]:
guess('graphics', count=10)

Graph,0.889238
printing,0.882980
Geometry,0.881459
Draw,0.875436
fps,0.859542
works,0.848875
coordinates,0.845141
Components,0.842674
interop,0.841272
Benchmark,0.840054


In [22]:
guess('deluge')

Flood,0.864981
foobar,0.825881
lambda-demo,0.819231
Dynamo,0.816781
yule,0.813927


In [23]:
guess('source control')

git,0.822978
Linkage,0.807133
gitscripts,0.806824
capture.net,0.803136
test-site,0.801779


In [24]:
guess('source-control')

git,0.831379
git-notes,0.830440
dot-imports,0.815820
gitscripts,0.812936
editor-configs,0.812632
